# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Our first Python Spark SQL example") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
path = "mini_sparkify_event_data.json"
user_log = spark.read.json(path)

In [4]:
user_log.createOrReplaceTempView("data")


In [5]:
#checking for any missing userId's or sessionId's, none
spark.sql('''
        select *
        from data
        where userId is null or sessionId is null
''').show()

#noticed that there were null songs, that is when page's value is not'next song'
spark.sql('''
        select distinct page
        from data
        where song is not null
''').show(40)

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+

+--------+
|    page|
+--------+
|NextSong|
+--------+



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [6]:
spark.udf.register("day", lambda x: str(datetime.datetime.fromtimestamp(x / 1000.0).day))
spark.udf.register("time", lambda x: str(datetime.datetime.fromtimestamp(x / 1000.0)))

<function __main__.<lambda>(x)>

In [7]:
pivot_data = spark.sql('''
        
        select userId, sessionId, page, count(page) as page_count
        from data
        where userId != ""
        group by userId, sessionId, page
        order by userId, sessionId, page_count desc
        

''')
pivot_data.createOrReplaceTempView("pivot_data")


In [8]:
transformed_data = spark.sql('''
            select userId, sessionId,
                sum(CancellationConfirmation) as CancellationConfirmation,
                sum(SubmitUpgrade) as SubmitUpgrade,
                sum(Upgrade) as Upgrade,
                sum(Help) as Help,
                sum(ThumbsUp) as ThumbsUp,
                sum(NextSong) as NextSong,
                sum(AddFriend) as AddFriend,
                sum(AddtoPlaylist) as AddtoPlaylist,
                sum(Settings) as Settings,
                sum(About) as About,
                sum(SaveSettings) as SaveSettings,
                sum(Logout) as Logout,
                sum(RollAdvert) as RollAdvert,
                sum(Downgrade) as Downgrade,
                sum(Home) as Home,
                sum(ThumbsDown) as ThumbsDown,
                sum(SubmitDowngrade) as SubmitDowngrade,
                sum(Cancel) as Cancel
                
                from (select pivot_data.*,
                    case when page = "Cancellation Confirmation" then page_count else 0 end as CancellationConfirmation,
                    case when page = "Submit Upgrade" then page_count  else 0 end as SubmitUpgrade,
                    case when page = "Upgrade" then page_count  else 0 end as Upgrade,
                    case when page = "Help" then page_count  else 0 end as Help,
                    case when page = "Thumbs Up" then page_count  else 0 end as ThumbsUp,
                    case when page = "NextSong" then page_count  else 0 end as NextSong,
                    case when page = "Add Friend" then page_count  else 0 end as AddFriend,
                    case when page = "Add to Playlist" then page_count  else 0 end as AddtoPlaylist,
                    case when page = "Settings" then page_count  else 0 end as Settings,
                    case when page = "About" then page_count  else 0 end as About,
                    case when page = "Save Settings" then page_count  else 0 end as SaveSettings,
                    case when page = "Logout" then page_count  else 0 end as Logout,
                    case when page = "Roll Advert" then page_count  else 0 end as RollAdvert,
                    case when page = "Downgrade" then page_count  else 0 end as Downgrade,
                    case when page = "Home" then page_count  else 0 end as Home,
                    case when page = "Thumbs Down" then page_count  else 0 end as ThumbsDown,
                    case when page = "Submit Downgrade" then page_count  else 0 end as SubmitDowngrade,
                    case when page = "Cancel" then page_count  else 0 end as Cancel
                from pivot_data) pivot_ext
                group by userId, sessionId
                order by userId, sessionId asc
''')
transformed_data.createOrReplaceTempView("transformed")

In [9]:
spark.sql('''
        
        select userId, count(*) as sessions
        FROM (select distinct userId, sessionId
                from data
                where userId != "")
        group by userId
        order by sessions Desc
        

''')
#There are matching sessionId's between 

DataFrame[userId: string, sessions: bigint]

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [10]:
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler

ignore = ['userId','sessionId',"CancellationConfirmation" ]

assembler = VectorAssembler(
  inputCols=[x for x in transformed_data.columns if x not in ignore], outputCol="features"
)

churn_assembler = VectorAssembler(
    inputCols =["CancellationConfirmation"], outputCol='churn'

)

assembled = churn_assembler.transform(assembler.transform(transformed_data))

In [11]:
assembled.toPandas()

,userId,sessionId,CancellationConfirmation,SubmitUpgrade,Upgrade,Help,ThumbsUp,NextSong,AddFriend,AddtoPlaylist,...,SaveSettings,Logout,RollAdvert,Downgrade,Home,ThumbsDown,SubmitDowngrade,Cancel,features,churn
0,10,9,0,0,0,0,3,57,3,1,...,0,1,0,0,4,0,0,0,"(0.0, 0.0, 0.0, 3.0, 57.0, 3.0, 1.0, 1.0, 0.0,...",[0.0]
1,10,595,0,0,0,0,23,381,5,7,...,0,4,0,3,16,4,0,0,"(0.0, 0.0, 0.0, 23.0, 381.0, 5.0, 7.0, 4.0, 2....",[0.0]
2,10,1047,0,0,0,0,2,21,1,0,...,0,0,0,0,1,0,0,0,"(0.0, 0.0, 0.0, 2.0, 21.0, 1.0, 0.0, 0.0, 0.0,...",[0.0]
3,10,1414,0,0,0,1,5,63,1,0,...,1,2,0,1,3,0,0,0,"(0.0, 0.0, 1.0, 5.0, 63.0, 1.0, 0.0, 1.0, 0.0,...",[0.0]
4,10,1592,0,0,0,0,1,67,1,0,...,0,2,1,2,4,0,0,0,"(0.0, 0.0, 0.0, 1.0, 67.0, 1.0, 0.0, 0.0, 0.0,...",[0.0]
5,10,1981,0,0,0,0,3,84,1,1,...,0,2,0,1,2,0,0,0,"(0.0, 0.0, 0.0, 3.0, 84.0, 1.0, 1.0, 1.0, 0.0,...",[0.0]
6,100,256,0,0,0,1,10,148,2,4,...,1,0,1,0,5,3,0,0,"[0.0, 0.0, 1.0, 10.0, 148.0, 2.0, 4.0, 1.0, 1....",[0.0]
7,100,369,0,0,0,0,6,91,4,1,...,0,1,0,1,3,1,0,0,"(0.0, 0.0, 0.0, 6.0, 91.0, 4.0, 1.0, 0.0, 0.0,...",[0.0]
8,100,435,0,0,0,0,0,15,0,1,...,0,0,0,0,1,0,0,0,"(0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 1.0, 0.0, 0.0,...",[0.0]
9,100,556,0,0,0,0,10,237,5,6,...,1,4,0,5,14,2,0,0,"[0.0, 0.0, 0.0, 10.0, 237.0, 5.0, 6.0, 2.0, 1....",[0.0]


In [19]:
binarizer = Binarizer(threshold=0.5, inputCol="churn", outputCol="churn2")

binarizedDataFrame = binarizer.transform(assembled)

In [45]:
final = binarizedDataFrame.select(['features','churn2'])


DataFrame[features: vector, churn2: vector]

In [29]:

splits = final.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

Row(features=SparseVector(17, {3: 3.0, 4: 57.0, 5: 3.0, 6: 1.0, 7: 1.0, 10: 1.0, 13: 4.0}), churn2=DenseVector([0.0]))

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [23]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol="features", labelCol='churn2')

# Fit the model
lrModel = lr.fit(train_df)

IllegalArgumentException: 'requirement failed: Column churn2 must be of type numeric but was actually of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>>.'

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.